# BGP Products - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
#import consts

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("Report Mylan 2019.pdf", pages='all', lattice=True)

## Format Table

In [11]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Save Type Index
index_hco = df_export[df_export['empty'].str.contains("H\rC\rO\rs", na=False)].index[0]

#Replace Strings
df_export.loc[df_export['empty'] == 'H\rC\rP\rs', 'empty'] = np.nan
df_export.loc[df_export['empty'] == 'H\rC\rO\rs', 'empty'] = np.nan

#Shift rows
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Add type
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Remove Carination
df_export = remove_carination(df_export, " ")

#Remove empty
df_export.drop(columns=['empty', 'empty1'], inplace=True)

#Remove Titles in Name
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Set Type -> Falsche Namen
#df_export = set_type_by_alphabetical_order(df_export)

#shift Surname/Lastname
df_export = revert_name(df_export, ' ')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'bgp')


saved


In [ ]:
#write_to_excel(df_export, 'tmp.xlsx', open=True)